# DECISION TREE FROM SCRATCH


In [14]:
import numpy as np
import pprint

class ID3:
  def fit(self, input, output):
    data = input.copy()
    data[output.name] = output
    pp = pprint.PrettyPrinter(indent=4)
    self.tree = self.decision_tree(data, data, input.columns, output.name)

    pp.pprint(self.tree)

  def predict(self, input):
    # convert input data into a dictionary of samples
    samples = input.to_dict(orient='records')
    predictions = []

    # makes prediction for all samples
    for sample in samples:
      predictions.append(self.make_prediction(sample, self.tree, 1.0))

    return predictions

  def entropy(self, attribute_column):
    # find unique values and their frequency counts for the given attribute
    values, counts = np.unique(attribute_column, return_counts=True)

    # calculate entropy for each unique value
    entropy_list = []

    for i in range(len(values)):
      probability = counts[i]/np.sum(counts)
      entropy_list.append(-probability*np.log2(probability))

    # calculate sum of individual entropy values
    total_entropy = np.sum(entropy_list)

    return total_entropy

  def information_gain(self, data, feature_attribute_name, target_attribute_name):
    # find total entropy of given subset
    total_entropy = self.entropy(data[target_attribute_name])

    # find unique values and their frequency counts for the attribute to be split
    values, counts = np.unique(data[feature_attribute_name], return_counts=True)

    # calculate weighted entropy of subset
    weighted_entropy_list = []

    for i in range(len(values)):
      subset_probability = counts[i]/np.sum(counts)
      subset_entropy = self.entropy(data.where(data[feature_attribute_name]==values[i]).dropna()[target_attribute_name])
      weighted_entropy_list.append(subset_probability*subset_entropy)

    total_weighted_entropy = np.sum(weighted_entropy_list)

    # calculate information gain
    information_gain = total_entropy - total_weighted_entropy

    return information_gain

  def decision_tree(self, data, orginal_data, feature_attribute_names, target_attribute_name, parent_node_class=None):
    # if data is pure, return the majority class of subset
    unique_classes = np.unique(data[target_attribute_name])
    if len(unique_classes) <= 1:
      return unique_classes[0]
    # if subset is empty, return majority class of original data
    elif len(data) == 0:
      majority_class_index = np.argmax(np.unique(original_data[target_attribute_name], return_counts=True)[1])
      return np.unique(original_data[target_attribute_name])[majority_class_index]
    # if data set contains no features to train with, return parent node class
    elif len(feature_attribute_names) == 0:
      return parent_node_class
    # if none of the above are true, construct a branch:
    else:
      # determine parent node class of current branch
      majority_class_index = np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])
      parent_node_class = unique_classes[majority_class_index]

      # determine information gain values for each feature
      # choose feature which best splits the data, ie. highest value
      ig_values = [self.information_gain(data, feature, target_attribute_name) for feature in feature_attribute_names]
      best_feature_index = np.argmax(ig_values)
      best_feature = feature_attribute_names[best_feature_index]

      # create tree structure, empty at first
      tree = {best_feature: {}}

      # remove best feature from available features, it will become the parent node
      feature_attribute_names = [i for i in feature_attribute_names if i != best_feature]

      # create nodes under parent node
      parent_attribute_values = np.unique(data[best_feature])
      for value in parent_attribute_values:
        sub_data = data.where(data[best_feature] == value).dropna()

        # call the algorithm recursively
        subtree = self.decision_tree(sub_data, orginal_data, feature_attribute_names, target_attribute_name, parent_node_class)

        # add subtree to original tree
        tree[best_feature][value] = subtree

      return tree

  def make_prediction(self, sample, tree, default=1):
    # map sample data to tree
    for attribute in list(sample.keys()):
      # check if feature exists in tree
      if attribute in list(tree.keys()):
        try:
          result = tree[attribute][sample[attribute]]
        except:
          return default

        result = tree[attribute][sample[attribute]]

        # if more attributes exist within result, recursively find best result
        if isinstance(result, dict):
          return self.make_prediction(sample, result)
        else:
          return result

# APPLYING ID3 ON DATASET

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv("diabetes_original.csv", header=None) 
df.head()

,0,1,2,3,4,5,6,7,8
0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0


In [16]:
columns = ['Pregnancies','Glucose', 'BloodPressure','SkinThickness','Insulin', 'BMI','DiabetesPedigreeFunction', 'Age','Outcome']
df.columns = columns
X = df.drop(columns="Outcome")
Y = df["Outcome"]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [18]:
model = ID3()
tree = model.fit(X_train, y_train)

{   'DiabetesPedigreeFunction': {   '0.078': '0',
                                    '0.085': '0',
                                    '0.088': {   'Pregnancies': {   '1': '1',
                                                                    '2': '0'}},
                                    '0.089': '0',
                                    '0.092': '0',
                                    '0.096': '0',
                                    '0.1': '0',
                                    '0.101': '0',
                                    '0.102': '0',
                                    '0.108': '0',
                                    '0.118': '0',
                                    '0.121': {   'Pregnancies': {   '3': '0',
                                                                    '6': '1'}},
                                    '0.122': '0',
                                    '0.123': '0',
                                    '0.126': '0',
                                    

In [19]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.22077922077922077

In [20]:
print('InfoGain of Pregnancies:',model.information_gain(df,df.columns[0],df.columns[:-1]))
print('InfoGain of Glucose:',model.information_gain(df,df.columns[1],df.columns[:-1]))
print('InfoGain of BloodPressure:',model.information_gain(df,df.columns[2],df.columns[:-1]))
print('InfoGain of SkinThickness:',model.information_gain(df,df.columns[3],df.columns[:-1]))
print('InfoGain of Insulin:',model.information_gain(df,df.columns[4],df.columns[:-1]))
print('InfoGain of BMI:',model.information_gain(df,df.columns[5],df.columns[:-1]))
print('InfoGain of DiabetesPedigreeFunction:',model.information_gain(df,df.columns[6],df.columns[:-1]))
print('InfoGain of Age:',model.information_gain(df,df.columns[7],df.columns[:-1]))

InfoGain of Pregnancies: 1.3216688531588368
InfoGain of Glucose: 3.1123929580193828
InfoGain of BloodPressure: 1.9690855943819807
InfoGain of SkinThickness: 1.9666259407736852
InfoGain of Insulin: 2.4128222616305264
InfoGain of BMI: 3.67113509683084
InfoGain of DiabetesPedigreeFunction: 4.5551430644977255
InfoGain of Age: 2.0981808082132583


In [21]:
df_2 = df.drop(['Insulin','BloodPressure','SkinThickness','Pregnancies','DiabetesPedigreeFunction'], axis=1)
df_2.head()


,Glucose,BMI,Age,Outcome
0,Glucose,BMI,Age,Outcome
1,148,33.6,50,1
2,85,26.6,31,0
3,183,23.3,32,1
4,89,28.1,21,0


In [22]:
columns = ['Glucose','BMI','Age','Outcome']
df_2.columns = columns
X = df_2.drop(columns="Outcome")
Y = df_2["Outcome"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
model_3 = ID3()
tree = model_3.fit(X_train, y_train)

{   'BMI': {   '0': {   'Glucose': {   '102': '0',
                                       '114': '0',
                                       '115': '1',
                                       '125': '1',
                                       '74': '0',
                                       '80': '0',
                                       '94': '0'}},
               '18.2': '0',
               '18.4': '0',
               '19.1': '0',
               '19.3': '0',
               '19.4': '0',
               '19.5': '0',
               '19.6': '0',
               '19.9': '0',
               '20': '0',
               '20.1': '0',
               '20.4': '0',
               '20.8': '0',
               '21': '0',
               '21.1': '0',
               '21.2': '0',
               '21.7': '0',
               '21.8': '0',
               '21.9': '0',
               '22.1': '0',
               '22.2': '0',
               '22.4': '0',
               '22.5': '0',
               '22.6': '0',
    

In [23]:
y_pred = model_3.predict(X_test)
accuracy_score(y_test, y_pred)

0.6501950585175552